In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Thesis_2023_24/

/content/drive/MyDrive/Thesis_2023_24


In [1]:
!pip install routing_transformer --upgrade

     ---------------------------------------- 42.2/42.2 kB 1.0 MB/s eta 0:00:00
  Attempting uninstall: einops
    Found existing installation: einops 0.3.0
    Uninstalling einops-0.3.0:
      Successfully uninstalled einops-0.3.0


In [2]:
!pip install packaging

In [7]:
# %%writefile setup.sh

!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" ./apex

Cloning into 'apex'...


Using pip 22.3.1 from D:\Programs\anaconda3\lib\site-packages\pip (python 3.10)
Processing d:\thesis_2023-24\codes\transformer-mm-explainability\apex
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


DEPRECATION: --no-binary currently disables reading from the cache of locally built wheels. In the future --no-binary will not influence the wheel cache. pip 23.1 will enforce this behaviour change. A possible replacement is to use the --no-cache-dir option. You can use the flag --use-feature=no-binary-enable-wheel-cache to test the upcoming behaviour. Discussion can be found at https://github.com/pypa/pip/issues/11453
  Running command pip subprocess to install build dependencies
  DEPRECATION: --no-binary currently disables reading from the cache of locally built wheels. In the future --no-binary will not influence the wheel cache. pip 23.1 will enforce this behaviour change. A possible replacement is to use the --no-cache-dir option. You can use the flag --use-feature=no-binary-enable-wheel-cache to test the upcoming behaviour. Discussion can be found at https://github.com/pypa/pip/issues/11453
       ---------------------------------------- 2.2/2.2 MB 6.7 MB/s eta 0:00:00
    Getti

In [4]:
# !sh setup.sh

'sh' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
from apex import amp
import torch
from routing_transformer import RoutingTransformerLM

SEQ_LEN = 131072

s = RoutingTransformerLM(
    num_tokens = 256,
    dim = 512,
    heads = 8,
    depth = 1,
    window_size = 128,
    reversible = True,
    ff_chunks = 40,
    max_seq_len = SEQ_LEN
).cuda()

s = amp.initialize(s, opt_level='O2')

x = torch.randint(0, 256, (1, SEQ_LEN)).cuda()
y, aux_loss = s(x)
y.sum().backward()

print('success')

ImportError: cannot import name 'amp' from 'apex' (unknown location)

In [ ]:
import torch
from routing_transformer import RoutingTransformerLM

model = RoutingTransformerLM(
    num_tokens = 20000,
    dim = 512,
    heads = 8,
    depth = 12,
    max_seq_len = 8192,
    causal = True,           # auto-regressive or not
    emb_dim = 128,           # embedding factorization, from Albert
    weight_tie = False,      # weight tie layers, from Albert
    tie_embedding = False,   # multiply final embeddings with token weights for logits
    dim_head = 64,           # be able to fix the dimension of each head, making it independent of the embedding dimension and the number of heads
    attn_dropout = 0.1,      # dropout after attention
    attn_layer_dropout = 0., # dropout after self attention layer
    ff_dropout = 0.1,        # feedforward dropout
    layer_dropout = 0.,      # layer dropout
    window_size = 128,       # target window size of each cluster
    n_local_attn_heads = 4,  # number of local attention heads
    reversible = True,       # reversible networks for memory savings, from Reformer paper
    ff_chunks = 10,          # feed forward chunking, from Reformer paper
    ff_glu = True,           # use GLU variant in feedforward
    pkm_layers = (4, 7),     # specify layers to use product key memory. paper shows 1 or 2 modules near the middle of the transformer is best
    pkm_num_keys = 128,      # defaults to 128, but can be increased to 256 or 512 as memory allows
    moe_layers = (3, 6),     # specify which layers to use mixture of experts
    moe_num_experts = 4,     # number of experts in the mixture of experts layer, defaults to 4. increase for adding more parameters to model
    moe_loss_coef = 1e-2,    # the weight for the auxiliary loss in mixture of experts to keep expert usage balanced
    num_mem_kv = 8,          # number of memory key/values to append to each cluster of each head, from the 'All-Attention' paper. defaults to 1 in the causal case for unshared QK to work
    use_scale_norm = False,  # use scale norm, simplified normalization from 'Transformers without Tears' paper
    use_rezero = False,      # use Rezero with no normalization
    shift_tokens = True      # shift tokens by one along sequence dimension, for a slight improvement in convergence
).cuda()

x = torch.randint(0, 20000, (1, 8192)).long().cuda()
input_mask = torch.ones_like(x).bool().cuda()

y, aux_loss = model(x, input_mask = input_mask) # (1, 8192, 20000)
aux_loss.backward() # add auxiliary loss to main loss before backprop

print(x)
print(input_mask)
print(y)
print(aux_loss)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1334: UserWarning: Using a non-full backward hook when outputs are generated by different autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when outputs are generated by different autograd Nodes "


tensor([[ 8853,  1834,   998,  ..., 15881, 10238,  7324]], device='cuda:0')
tensor([[True, True, True,  ..., True, True, True]], device='cuda:0')
tensor([[[ 0.1505,  0.4908, -0.2077,  ..., -0.1255,  0.8346,  0.5536],
         [ 0.9205, -0.0577,  0.2428,  ..., -1.0441,  1.2415,  0.0124],
         [ 0.0297, -0.5290,  0.1574,  ..., -0.6887,  1.1949,  0.2831],
         ...,
         [-0.1320, -0.8083,  0.6914,  ..., -0.6083,  0.1129, -1.0397],
         [ 0.1453, -0.6838,  0.9102,  ..., -0.3746,  0.2540, -0.9866],
         [ 0.3760, -0.8628,  0.8975,  ..., -0.9756,  0.2434, -1.4790]]],
       device='cuda:0', grad_fn=<ViewBackward0>)
tensor(0.0502, device='cuda:0', grad_fn=<AddBackward0>)
